中文分词主要有归纳为三种：规则分词、统计分词和混合分词（规则+统计）。
一、规则分词：一种机械的分词方法，主要是通过维护词典，在切分语句时，将语句的每个字符串与词表中的词进行逐一匹配，找到则切分，否则  不予切分。按照匹配切分的方式主要有三种方法：正向最大匹配法、逆向最大匹配法和双向最大匹配法。
1、正向最大匹配法（Maximum Match Method, MM法）：假定分词词典中的最长词有i个汉字字符，则用被处理文档的当前字串中的前i个字作为匹配字段，查找字典。若字典中存在这样的一个i字词，则匹配成功，匹配字段被作为一个词切分出来，如果词典中找不到这样的一个i字词，则匹配失败，将匹配字段中的最后一个字去掉，对剩下的字串重新进行匹配处理，如此进行下去，直到匹配成功（即切分出一个词或剩余字串的长度为零为止）这样完成了一轮匹配，然后取下一个i字字串进行匹配处理，直到文档被扫描完为止。
2、逆向最大匹配法（Reverse Maximum Match Method, RMM法）：和MM法基本原理相同，不同的是分词切分的方向和MM法相反，逆向最大匹配法从处理文档的末端开始匹配扫描，每次去最末端的i个字符作为匹配字段，若匹配失败，则去掉匹配字段最前面的一个字，继续匹配。相应的它使用的分词词典是逆序词典，其中的每个词条都将逆序方式存放。在实际处理时，先将文档进行倒排处理，生成逆序文档。然后根据逆序词典，对逆序文档用正向最大匹配法处理即可。
3、双向最大匹配法（Bi-direction Matching method）：将正向最大匹配法得到的分词结果和逆向最大匹配法得到的结果进行比较，然后按照最大匹配原则选取词数切分最少的作为结果。其匹配规则是(1)如果正逆向分词结果词数不同，则取分词数量较少的那个；(2)如果分词结果词数相同，且分词结果也相同，则返回任意一个结果都可以，若分词结果不同，则返回单字较少的那个分词结果。
\[问题：若单字的词数也相同该若何返回结果？\]
基于规则的分词，一般都较为简单高效，但是词典的维护是一个很庞大的工程，并且现在也很难覆盖所有的词。

In [6]:
# 正向最大匹配法
class mm(object):
    def __init__(self):
        self.window_size = 3
    def cut(self, text):
        result = []
        index = 0
        text_length = len(text)
        dic = ['研究', '研究生', '的', '生命', '命', '起源']
        while text_length > index:
            for size in range(self.window_size + index, index, -1):
                piece = text[index:size]
                if piece in dic:
                    index = size - 1
                    break
            index = index + 1
            result.append(piece+ '---')
        return result
text = '研究生命的起源'
tokenizer = mm()
print(tokenizer.cut(text))

['研究生---', '命---', '的---', '起源---']


In [21]:
# 逆序最大匹配法
class rmm(object):
    def __init__(self):
        self.window_size = 3
    def cut(self, text):
        result = []
        index = len(text)
        dic = ['研究', '研究生', '的', '生命', '命', '起源']
        while index > 0:
            for size in range(index - self.window_size, index):
                piece = text[size: index]
                if piece in dic:
                    index = size
                    break
            result.append(piece + '---')
        result.reverse()
        return result

text = '研究生命的起源'
tokenize = rmm()
print(tokenize.cut(text))

['研究---', '生命---', '的---', '起源---']


In [22]:
# 逆序最大匹配法
# 词典的存储方式是逆序的，将文档逆序处理之后按照正向最大匹配法实现
class rmm_(object):
    def __init__(self):
        self.window_size = 3
    def cut(self, text):
        result = []
        index = 0
        text_len = len(text)
        text = text[::-1]
        dic = ['究研', '生究研', '的', '命生', '命', '源起']
        while text_len > index:
            for size in range(self.window_size + index, index, -1):
                piece = text[index:size]
                if piece in dic:
                    index = size - 1
                    break
            index = index + 1
            result.append(piece[::-1] + '---')
            result.reverse()
        return result

text = '研究生命的起源'
tokenize = rmm_()
print(tokenize.cut(text))

['研究---', '的---', '起源---', '生命---']


In [29]:
# 双向最大匹配法
class BiM(object):
    def __init__(self):
        self.window_size = 3
    def cut(self, text):
        dic = ['研究', '研究生', '的', '生命', '命', '起源']
        # 正向最大匹配
        result = {'count':0, 'ret':[]}
        index = 0
        text_len = len(text)
        while text_len > index:
            for size in range(index + self.window_size, index, -1):
                piece = text[index: size]
                if piece in dic:
                    index = size - 1
                    break
            if len(piece) == 1:
                result['count'] = result['count'] + 1
            result['ret'].append(piece + '---')
            index = index + 1
        # 逆向最大匹配
        result_ = {'count':0, 'ret':[]}
        index = text_len
        while index > 0:
            for size in range(index - self.window_size, index):
                piece = text[size: index]
                if piece in dic:
                    index = size
                    break
            if len(piece) == 1:
                result_['count'] = result_['count'] + 1
            result_['ret'].append(piece + '---')
        result_['ret'].reverse()
        if len(result['ret']) > len(result_['ret']):
            return result_['ret']
        elif len(result['ret']) < len(result_['ret']):
            return result['ret']
        else:
            if result['count'] < result_['count']:
                return result['ret']
            else:
                return result_['ret']

text = '研究生命的起源'
tokenizer = BiM()
print(tokenizer.cut(text))

['研究---', '生命---', '的---', '起源---']


二、统计分词：随着大规模语料库的建立，统计机器学习方法的研究和发展，基于统计的中文分词算法渐渐成为主流。
其思想是把每个词看做是由词的最小单位的各个字组成的，如果相连的字在不同的文本中出现的次数越多，就证明这相连的字很可能就是一个词。因此就可以利用字与字相邻出现的频率来反应成词的可靠度，统计语料中相邻共现的各个字的组合的频度，当组合频度高于某一个临界值时，便可认为此字组可能会构成一个词语。
统计分词一般有两步：1、建立统计语言模型；2、对句子进行单词划分，然后对划分结果进行概率计算，获得概率最大的分词方式。统计学习算法有隐含马尔科夫（HMM）、条件随机场（CRF）等。
1、隐含马尔科夫模型（HMM）：将分词作为字在字串中的序列标注任务来实现的。其基本思路是每个字在构造一个特定的词语时都占据着一个确定的构词位置（即词位），现规定每个字最多只有四个构词位置。
